In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table
from configparser import ConfigParser
from sqlalchemy import text
import csv
import pandas as pd
from difflib import SequenceMatcher

In [2]:
config = ConfigParser()
config.read("setting.ini")
dbsett = config["eurodeer_db"]
networks = ["eurodeer", "euroboar", "eurowildcat", "euroibex", "eurolynx",]

For each network we check the lookup tables and save the output of the records as a string in a dictionary

In [3]:
output = {}
for net in networks:
    dbname = "{net}_db".format(net=net)
    db_connection_url = "postgresql://{us}:{pas}@{host}:{port}/{db}".format(us=dbsett['user'],
                                                                      pas=dbsett['password'],
                                                                      host=dbsett['host'],
                                                                      port=dbsett['port'],
                                                                      db=dbname
                                                                     )
    con = create_engine(db_connection_url)
    connection = con.connect()
    inspector = inspect(con)
    tables = inspector.get_table_names(schema="lu_tables")
    for tab in tables:
        meta = MetaData(schema="lu_tables")
        meta.reflect(bind=con)
        primary_col = meta.tables[f"lu_tables.{tab}"].primary_key.columns.values()[0].name
        if tab not in output.keys():
            output[tab] = {}           
        results = pd.read_sql_query(f"select * from lu_tables.{tab} order by {primary_col}", con)
        output[tab][net] = results.to_csv(index=False, sep=";")
    

We execute the comparison of previous outputs, for each tables we check between the networks. We calculate the ratio of differences and save them in a dictionary

In [4]:
results = {"tests": ["eurodeer-euroboar", "eurodeer-eurowildcat", "eurodeer-eurolynx", "euroboar-eurowildcat", "euroboar-eurolynx", "eurowildcat-eurolynx"]}
for tab in output.keys():
    out = []
    # eurodeer-euroboar
    if "eurodeer" in output[tab].keys() and "euroboar" in output[tab].keys():
        seq_match = SequenceMatcher(None, output[tab]["eurodeer"], output[tab]["euroboar"])
        out.append(seq_match.ratio())
    elif "eurodeer" not in output[tab].keys() and "euroboar" not in output[tab].keys():
        out.append(None)
    elif "eurodeer" not in output[tab].keys():
        out.append(-1)
    else:
        out.append(-2)
    # eurodeer-eurowildcat
    if "eurodeer" in output[tab].keys() and "eurowildcat" in output[tab].keys():
        seq_match = SequenceMatcher(None, output[tab]["eurodeer"], output[tab]["eurowildcat"])
        out.append(seq_match.ratio())
    elif "eurodeer" not in output[tab].keys() and "eurowildcat" not in output[tab].keys():
        out.append(None)
    elif "eurodeer" not in output[tab].keys():
        out.append(-1)
    else:
        out.append(-2)
    # eurodeer-eurolynx
    if "eurodeer" in output[tab].keys() and "eurolynx" in output[tab].keys():
        seq_match = SequenceMatcher(None, output[tab]["eurodeer"], output[tab]["eurolynx"])
        out.append(seq_match.ratio())
    elif "eurodeer" not in output[tab].keys() and "eurolynx" not in output[tab].keys():
        out.append(None)
    elif "eurodeer" not in output[tab].keys():
        out.append(-1)
    else:
        out.append(-2)
    # euroboar - eurowildcat
    if "euroboar" in output[tab].keys() and "eurowildcat" in output[tab].keys():
        seq_match = SequenceMatcher(None, output[tab]["euroboar"], output[tab]["eurowildcat"])
        out.append(seq_match.ratio())
    elif "euroboar" not in output[tab].keys() and "eurowildcat" not in output[tab].keys():
        out.append(None)
    elif "euroboar" not in output[tab].keys():
        out.append(-1)
    else:
        out.append(-2)
    # euroboar - eurolynx
    if "euroboar" in output[tab].keys() and "eurolynx" in output[tab].keys():
        seq_match = SequenceMatcher(None, output[tab]["euroboar"], output[tab]["eurolynx"])
        out.append(seq_match.ratio())
    elif "euroboar" not in output[tab].keys() and "eurolynx" not in output[tab].keys():
        out.append(None)
    elif "euroboar" not in output[tab].keys():
        out.append(-1)
    else:
        out.append(-2)
    # eurowildcat - eurolynx
    if "eurolynx" in output[tab].keys() and "eurowildcat" in output[tab].keys():
        seq_match = SequenceMatcher(None, output[tab]["eurolynx"], output[tab]["eurowildcat"])
        out.append(seq_match.ratio())
    elif "eurolynx" not in output[tab].keys() and "eurowildcat" not in output[tab].keys():
        out.append(None)
    elif "eurolynx" not in output[tab].keys():
        out.append(-2)
    else:
        out.append(-1)
    results[tab] = out

We add the dictionary to a Pandas dataframe

In [5]:
results_pd = pd.DataFrame.from_dict(results, orient="index")
results_pd.columns = results_pd.iloc[0]
results_pd = results_pd[1:]
results_pd.reset_index(inplace=True)

Print the dataframe setting a text gradient according the values. 

Values meaning:
* None: that table is missing in both the network database
* -1: the table is missing in the first network of comparison. For example in the column eurodeer-euroboar -1 means that table is missing in eurodeer
* -2:  the table is missing in the second network of comparison. For example in the column eurodeer-euroboar -2 means that table is missing in euroboar

In [6]:
results_pd.style.text_gradient(cmap="RdYlGn", vmin=0, vmax=1, subset=["eurodeer-euroboar", "eurodeer-eurowildcat", "eurodeer-eurolynx", "euroboar-eurowildcat", "euroboar-eurolynx", "eurowildcat-eurolynx"])

tests,index,eurodeer-euroboar,eurodeer-eurowildcat,eurodeer-eurolynx,euroboar-eurowildcat,euroboar-eurolynx,eurowildcat-eurolynx
0,lu_mortality_new,0.828311,-2,-2,-2,-2,None
1,lu_mortality,0.557742,0.064381,0.231534,0.042829,0.277924,0.040423
2,lu_action,-2,-2,-2,None,None,None
3,lu_activity_sensor_mode,0.918100,-2,0.203822,-2,0.212625,-1
4,lu_activity_validity,1.000000,-2,0.989610,-2,0.989610,-1
5,lu_age_class,0.413437,0.560440,0.650177,0.323450,0.306905,0.589091
6,lu_age_class_reddeer,-2,-2,-2,None,None,None
7,lu_behavior_handling,1.000000,0.195212,1.000000,0.195212,1.000000,0.195212
8,lu_behavior_release,1.000000,0.274933,1.000000,0.274933,1.000000,0.274933
9,lu_capture_methods,1.000000,0.678679,0.444954,0.678679,0.444954,0.783158
